In [7]:
import pandas as pd
import os
import glob

finance_data_dir = r"D:\CnnTA\v2\data_finance\train\1d"
sentiment_data_dir = r"D:\CnnTA\v2\data_sentim\train"

data_dirs = [finance_data_dir, sentiment_data_dir]

In [8]:
finance_symbols = []
sentiment_companies = []
all_csv_files = []

for data_dir in data_dirs:
    csv_files = glob.glob(os.path.join(data_dir, "*.csv"))
    all_csv_files.extend(csv_files)
    for file in csv_files:
        filename = os.path.basename(file)
        parts = filename.split('_')
        if filename.startswith('gdelt_'):
            # Sentiment: gdelt_company_...csv
            # Try to extract company name between 'gdelt_' and the last date part
            if len(parts) > 2:
                company = '_'.join(parts[1:-4]) if len(parts) > 5 else parts[1]
            else:
                company = parts[1] if len(parts) > 1 else ''
            sentiment_companies.append(company)
        else:
            # Finance: SYMBOL_...csv
            symbol = parts[0]
            finance_symbols.append(symbol)

print("Finance symbols:", set(finance_symbols))
print("Sentiment companies:", set(sentiment_companies))

Finance symbols: {'HD', 'PG', 'TRV', 'MCD', 'MSFT', 'MRK', 'VZ', 'GE', 'AXP', 'V', 'DD', 'CVX', 'DIS', 'GS', 'IBM', 'XOM', 'INTC', 'PFE', 'BA', 'AAPL', 'CSCO', 'JPM', 'KO', 'UNH', 'JNJ', 'NKE', 'CAT', 'MMM', 'WMT'}
Sentiment companies: {'travelers', 'boeing', 'chevron', 'apple', 'microsoft', 'ibm', 'nvidia', 'salesforce', 'nike', '3m', 'disney', 'cisco', 'honeywell', 'amazon', 'verizon', 'jpmorgan', 'american', 'caterpillar', "mcdonald's", 'visa', 'unitedhealth', 'goldman', 'home', 'walmart', 'amgen'}


In [9]:
# Mapping finance symbols to sentiment companies (where possible)
finance_to_sentiment = {
    'AAPL': 'apple',
    'AXP': 'american',
    'BA': 'boeing',
    'CAT': 'caterpillar',
    'CSCO': 'cisco',
    'CVX': 'chevron',
    'DIS': 'disney',
    'GS': 'goldman',
    'HD': 'home',
    'IBM': 'ibm',
    'JPM': 'jpmorgan',
    'MCD': "mcdonald's",
    'MMM': '3m',
    'MSFT': 'microsoft',
    'NKE': 'nike',
    'TRV': 'travelers',
    'UNH': 'unitedhealth',
    'V': 'visa',
    'VZ': 'verizon',
    'WMT': 'walmart',
}

# Reverse mapping: sentiment company to finance symbol (where possible)
sentiment_to_finance = {v: k for k, v in finance_to_sentiment.items() if v is not None}

In [10]:
for symbol in finance_symbols:
    # Find corresponding sentiment company name
    sentiment_company = finance_to_sentiment.get(symbol)
    if not sentiment_company:
        print(f"{symbol}: No sentiment mapping.")
        continue

    # Find finance file
    finance_file = None
    for f in all_csv_files:
        if os.path.basename(f).startswith(symbol + "_"):
            finance_file = f
            break
    if not finance_file:
        print(f"{symbol}: Finance file not found.")
        continue

    # Find sentiment file
    sentiment_file = None
    for f in all_csv_files:
        if os.path.basename(f).startswith(f"gdelt_{sentiment_company}_"):
            sentiment_file = f
            break
    if not sentiment_file:
        print(f"{symbol}: Sentiment file not found for {sentiment_company}.")
        continue

    # Read both files
    try:
        df_fin = pd.read_csv(finance_file)
        df_sent = pd.read_csv(sentiment_file)
    except Exception as e:
        print(f"{symbol}: Error reading files: {e}")
        continue

    # Try to find the date column (case-insensitive)
    fin_date_col = next((col for col in df_fin.columns if col.lower() == "date"), None)
    sent_date_col = next((col for col in df_sent.columns if col.lower() == "date"), None)

    if not fin_date_col or not sent_date_col:
        print(f"{symbol}: Date column not found in one of the files.")
        continue

    # Convert to datetime, handling Unix ms timestamps
    def parse_dates(series):
        # If all values are numbers or digit strings, treat as ms since epoch
        if pd.api.types.is_numeric_dtype(series) or series.astype(str).str.isdigit().all():
            return pd.to_datetime(series, unit='ms')
        else:
            return pd.to_datetime(series)

    try:
        fin_dates = parse_dates(df_fin[fin_date_col])
        sent_dates = parse_dates(df_sent[sent_date_col])
    except Exception as e:
        print(f"{symbol}: Error parsing dates: {e}")
        continue

    print(f"{symbol} ({sentiment_company}):")
    print(f"  Finance date range:  {fin_dates.min()}  to  {fin_dates.max()}")
    print(f"  Sentiment date range: {sent_dates.min()}  to  {sent_dates.max()}")

AAPL (apple):
  Finance date range:  2017-01-03 00:00:00  to  2022-12-30 00:00:00
  Sentiment date range: 2017-03-01 00:00:00  to  2022-12-31 00:00:00
AXP (american):
  Finance date range:  2017-01-03 00:00:00  to  2022-12-30 00:00:00
  Sentiment date range: 2017-02-01 00:00:00  to  2022-12-31 00:00:00
BA (boeing):
  Finance date range:  2017-01-03 00:00:00  to  2022-12-30 00:00:00
  Sentiment date range: 2017-01-13 00:00:00  to  2022-12-31 00:00:00
CAT (caterpillar):
  Finance date range:  2017-01-03 00:00:00  to  2022-12-30 00:00:00
  Sentiment date range: 2017-02-06 00:00:00  to  2022-12-31 00:00:00
CSCO (cisco):
  Finance date range:  2017-01-03 00:00:00  to  2022-12-30 00:00:00
  Sentiment date range: 2017-09-21 00:00:00  to  2022-12-31 00:00:00
CVX (chevron):
  Finance date range:  2017-01-03 00:00:00  to  2022-12-30 00:00:00
  Sentiment date range: 2017-02-08 00:00:00  to  2022-12-31 00:00:00
DD: No sentiment mapping.
DIS (disney):
  Finance date range:  2017-01-03 00:00:00  to 

In [11]:
# Check alignment between finance and sentiment CSVs for each symbol
for symbol in finance_symbols:
    sentiment_company = finance_to_sentiment.get(symbol)
    if not sentiment_company:
        continue

    # Find finance and sentiment files
    finance_file = None
    sentiment_file = None
    for f in all_csv_files:
        if os.path.basename(f).startswith(symbol + "_"):
            finance_file = f
        if os.path.basename(f).startswith(f"gdelt_{sentiment_company}_"):
            sentiment_file = f
    if not finance_file or not sentiment_file:
        continue

    try:
        df_fin = pd.read_csv(finance_file)
        df_sent = pd.read_csv(sentiment_file)
    except Exception as e:
        print(f"{symbol}: Error reading files: {e}")
        continue

    # Find date columns
    fin_date_col = next((col for col in df_fin.columns if col.lower() == "date"), None)
    sent_date_col = next((col for col in df_sent.columns if col.lower() == "date"), None)
    if not fin_date_col or not sent_date_col:
        print(f"{symbol}: Date column not found.")
        continue

    # Parse dates
    def parse_dates(series):
        if pd.api.types.is_numeric_dtype(series) or series.astype(str).str.isdigit().all():
            return pd.to_datetime(series, unit='ms')
        else:
            return pd.to_datetime(series)
    try:
        fin_dates = parse_dates(df_fin[fin_date_col])
        sent_dates = parse_dates(df_sent[sent_date_col])
    except Exception as e:
        print(f"{symbol}: Error parsing dates: {e}")
        continue

    # Set as index for fast lookup
    fin_dates_set = set(fin_dates.dt.normalize())
    sent_dates_set = set(sent_dates.dt.normalize())

    missing_in_sent = fin_dates_set - sent_dates_set
    extra_in_sent = sent_dates_set - fin_dates_set

    print(f"\n{symbol} ({sentiment_company}):")
    print(f"  Finance rows: {len(fin_dates_set)}")
    print(f"  Sentiment rows: {len(sent_dates_set)}")
    print(f"  Days in finance but missing in sentiment: {len(missing_in_sent)}")
    if missing_in_sent:
        print(f"    Missing days: {sorted(list(missing_in_sent))[:10]}{' ...' if len(missing_in_sent)>10 else ''}")
    print(f"  Days in sentiment but not in finance: {len(extra_in_sent)}")
    if extra_in_sent:
        print(f"    Extra days: {sorted(list(extra_in_sent))[:10]}{' ...' if len(extra_in_sent)>10 else ''}")
    print(f"  Overlap days: {len(fin_dates_set & sent_dates_set)}")



AAPL (apple):
  Finance rows: 1510
  Sentiment rows: 2132
  Days in finance but missing in sentiment: 39
    Missing days: [Timestamp('2017-01-03 00:00:00'), Timestamp('2017-01-04 00:00:00'), Timestamp('2017-01-05 00:00:00'), Timestamp('2017-01-06 00:00:00'), Timestamp('2017-01-09 00:00:00'), Timestamp('2017-01-10 00:00:00'), Timestamp('2017-01-11 00:00:00'), Timestamp('2017-01-12 00:00:00'), Timestamp('2017-01-13 00:00:00'), Timestamp('2017-01-17 00:00:00')] ...
  Days in sentiment but not in finance: 661
    Extra days: [Timestamp('2017-03-04 00:00:00'), Timestamp('2017-03-05 00:00:00'), Timestamp('2017-03-11 00:00:00'), Timestamp('2017-03-12 00:00:00'), Timestamp('2017-03-18 00:00:00'), Timestamp('2017-03-19 00:00:00'), Timestamp('2017-03-25 00:00:00'), Timestamp('2017-03-26 00:00:00'), Timestamp('2017-04-01 00:00:00'), Timestamp('2017-04-02 00:00:00')] ...
  Overlap days: 1471

AXP (american):
  Finance rows: 1510
  Sentiment rows: 2160
  Days in finance but missing in sentiment: 

In [14]:
from collections import Counter

# Consolidate sentiment-only days (e.g., weekends) to the next available finance day
# and count how many days are consolidated


# Sort the sets for ordered processing
missing_in_fin_sorted = sorted(missing_in_fin)
fin_dates_sorted = sorted(fin_dates_set)

# Build a lookup for the next available finance day
consolidation_map = {}
consolidated_count = 0

for sent_day in missing_in_fin_sorted:
    # Find the next finance day after the sentiment-only day
    next_fin_days = [d for d in fin_dates_sorted if d > sent_day]
    if next_fin_days:
        next_fin_day = next_fin_days[0]
        consolidation_map[sent_day] = next_fin_day
        consolidated_count += 1

print(f"Total sentiment-only days consolidated to next finance day: {consolidated_count}")
# Optionally, show a few examples
print("First 10 consolidated days (sentiment day -> next finance day):")
for i, (k, v) in enumerate(consolidation_map.items()):
    if i >= 10:
        break
    print(f"{k.date()} -> {v.date()}")

Total sentiment-only days consolidated to next finance day: 54
First 10 consolidated days (sentiment day -> next finance day):
2017-01-16 -> 2017-01-17
2017-02-20 -> 2017-02-21
2017-04-14 -> 2017-04-17
2017-05-29 -> 2017-05-30
2017-07-04 -> 2017-07-05
2017-09-04 -> 2017-09-05
2017-11-23 -> 2017-11-24
2017-12-25 -> 2017-12-26
2018-01-01 -> 2018-01-02
2018-01-15 -> 2018-01-16
